In [1]:
import os 
import cv2
import glob
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from keras.layers import Dense
from keras.models import Sequential
from keras.preprocessing import image
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model
from keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.applications.vgg19 import VGG19
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications.inception_v3 import InceptionV3
from tensorflow.keras.applications.inception_v3 import preprocess_input
from keras.layers import Convolution2D,Dense,MaxPool2D,Activation,Dropout,Flatten
from tensorflow.keras.callbacks import LearningRateScheduler
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalMaxPooling2D
     

In [4]:
def get_files(directory):
  if not os.path.exists(directory):
    return 0
  count=0
  # crawls inside folders
  for current_path,dirs,files in os.walk(directory):
    for dr in dirs:
      count+= len(glob.glob(os.path.join(current_path,dr+"/*")))
  return count
train_dir =r"C:\food\pythonProject\image data\potato\train"
test_dir=r"C:\food\pythonProject\image data\potato\val"

In [5]:
 #train file image count
train_samples =get_files(train_dir)
#to get tags
num_classes=len(glob.glob(train_dir+"/*")) 
#test file image count
test_samples=get_files(test_dir)
print(num_classes,"Classes")
print(train_samples,"Train images")
print(test_samples,"Test images")

3 Classes
1721 Train images
431 Test images


In [6]:

train_datagen=ImageDataGenerator(
    rescale=1./255,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True
    )
test_datagen=ImageDataGenerator(rescale=1./255)

In [8]:
input_shape=(256,256,3)
train_generator =train_datagen.flow_from_directory(train_dir,target_size=(256,256),batch_size=32)
test_generator=test_datagen.flow_from_directory(test_dir,shuffle=True,target_size=(256,256),batch_size=32)
     
     

Found 1721 images belonging to 3 classes.
Found 431 images belonging to 3 classes.


In [11]:
validation_generator = train_datagen.flow_from_directory(
                       test_dir, # same directory as training data
                       target_size=(256,256),
                       batch_size=32)

Found 431 images belonging to 3 classes.


fully fine-tuned model

In [27]:
import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Define hyperparameters
learning_rate = 5e-5
train_batch_size = 256
eval_batch_size = 256
num_epochs = 50

# Load pretrained VGG16 model
pretrained_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

# Freeze all layers in the pretrained model
for layer in pretrained_model.layers:
    layer.trainable = False

# Add custom classification head
x = Flatten()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model
optimizer = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999, epsilon=1e-08)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define learning rate scheduler
lr_scheduler = tf.keras.callbacks.LearningRateScheduler(
    lambda epoch: learning_rate * (1 - epoch / num_epochs),
    verbose=1
)

# Train the model
history = model.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // train_batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // eval_batch_size,
    verbose=1,
    callbacks=[lr_scheduler]
)



Epoch 1: LearningRateScheduler setting learning rate to 5e-05.
Epoch 1/50
8/8 [==============================] - 35s 4s/step - loss: 0.9464 - accuracy: 0.5391 - val_loss: 0.6287 - val_accuracy: 0.7812 - lr: 5.0000e-05

Epoch 2: LearningRateScheduler setting learning rate to 4.9e-05.
Epoch 2/50
8/8 [==============================] - 37s 5s/step - loss: 0.7308 - accuracy: 0.6836 - val_loss: 0.5329 - val_accuracy: 0.7188 - lr: 4.9000e-05

Epoch 3: LearningRateScheduler setting learning rate to 4.8e-05.
Epoch 3/50
8/8 [==============================] - 36s 4s/step - loss: 0.5379 - accuracy: 0.7845 - val_loss: 0.4193 - val_accuracy: 0.8438 - lr: 4.8000e-05

Epoch 4: LearningRateScheduler setting learning rate to 4.7e-05.
Epoch 4/50
8/8 [==============================] - 39s 5s/step - loss: 0.4992 - accuracy: 0.8086 - val_loss: 0.3573 - val_accuracy: 0.9062 - lr: 4.7000e-05

Epoch 5: LearningRateScheduler setting learning rate to 4.600000000000001e-05.
Epoch 5/50
8/8 [======================

fine tuned upto a specific layer

In [15]:
learning_rate = 5e-5
train_batch_size = 256
eval_batch_size = 256
num_epochs = 30
num_classes = 3  # Replace with the actual number of classes in your dataset

# Load pretrained VGG16 model
pretrained_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

# Freeze all layers up to a specific layer
freeze_until_layer = 'block4_conv3'
for layer in pretrained_model.layers:
    if layer.name == freeze_until_layer:
        break
    layer.trainable = False

# Add custom classification head
x = Flatten()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model2 = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model
optimizer = Adam(learning_rate=learning_rate)
model2.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Define learning rate scheduler
def lr_scheduler(epoch, lr):
    return lr * (1 - epoch / num_epochs)

lr_callback = LearningRateScheduler(lr_scheduler)

# Train the model
history = model2.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // train_batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // eval_batch_size,
    verbose=1,
    callbacks=[lr_callback]
)


Epoch 1/30
6/6 [==============================] - 70s 11s/step - loss: 0.9351 - accuracy: 0.5469 - val_loss: 0.7199 - val_accuracy: 0.8125 - lr: 5.0000e-05
Epoch 2/30
6/6 [==============================] - 63s 11s/step - loss: 0.5086 - accuracy: 0.8646 - val_loss: 0.3229 - val_accuracy: 0.9062 - lr: 4.8333e-05
Epoch 3/30
6/6 [==============================] - 63s 11s/step - loss: 0.4753 - accuracy: 0.8385 - val_loss: 0.3710 - val_accuracy: 0.8750 - lr: 4.5111e-05
Epoch 4/30
6/6 [==============================] - 61s 10s/step - loss: 0.3467 - accuracy: 0.8594 - val_loss: 0.3839 - val_accuracy: 0.8125 - lr: 4.0600e-05
Epoch 5/30
6/6 [==============================] - 62s 11s/step - loss: 0.2414 - accuracy: 0.9323 - val_loss: 0.1414 - val_accuracy: 0.9688 - lr: 3.5187e-05
Epoch 6/30
6/6 [==============================] - 61s 10s/step - loss: 0.3243 - accuracy: 0.8333 - val_loss: 0.1394 - val_accuracy: 0.9375 - lr: 2.9322e-05
Epoch 7/30
6/6 [==============================] - 61s 10s/step -

Using SGD as optimiser

In [13]:
from tensorflow.keras.optimizers import SGD

# Define hyperparameters
initial_learning_rate = 0.01
train_batch_size = 256
eval_batch_size = 256
num_epochs = 30
num_classes = 3  # Replace with the actual number of classes in your dataset

# Define learning rate scheduler
def lr_scheduler(epoch, lr):
    if epoch < 3:
        return lr * 10  # Increase learning rate by 10x for the first 3 epochs
    elif epoch < 7:
        return lr * 0.1  # Decrease learning rate by 10x for the next 4 epochs
    else:
        return lr * 0.01  # Decrease learning rate by 100x for the remaining epochs

lr_callback = LearningRateScheduler(lr_scheduler)

# Load pretrained VGG16 model
pretrained_model = VGG16(
    include_top=False,
    weights='imagenet',
    input_shape=(256, 256, 3)
)

# Freeze all layers up to a specific layer
freeze_until_layer = 'block4_conv3'
for layer in pretrained_model.layers:
    if layer.name == freeze_until_layer:
        break
    layer.trainable = False

# Add custom classification head
x = Flatten()(pretrained_model.output)
x = Dense(256, activation='relu')(x)
predictions = Dense(num_classes, activation='softmax')(x)

# Create the final model
model2 = Model(inputs=pretrained_model.input, outputs=predictions)

# Compile the model
optimizer = SGD(learning_rate=initial_learning_rate)
model2.compile(optimizer=optimizer,
               loss='categorical_crossentropy',
               metrics=['accuracy'])

# Train the model
history = model2.fit(
    train_generator,
    epochs=num_epochs,
    steps_per_epoch=train_generator.samples // train_batch_size,
    validation_data=validation_generator,
    validation_steps=validation_generator.samples // eval_batch_size,
    verbose=1,
    callbacks=[lr_callback]
)


Epoch 1/30
6/6 [==============================] - 52s 9s/step - loss: nan - accuracy: 0.5000 - val_loss: nan - val_accuracy: 0.5000 - lr: 0.1000
Epoch 2/30
6/6 [==============================] - 59s 10s/step - loss: nan - accuracy: 0.4635 - val_loss: nan - val_accuracy: 0.6250 - lr: 1.0000
Epoch 3/30
6/6 [==============================] - 61s 10s/step - loss: nan - accuracy: 0.4583 - val_loss: nan - val_accuracy: 0.3438 - lr: 10.0000
Epoch 4/30
6/6 [==============================] - 60s 10s/step - loss: nan - accuracy: 0.4583 - val_loss: nan - val_accuracy: 0.3750 - lr: 1.0000
Epoch 5/30
6/6 [==============================] - 65s 11s/step - loss: nan - accuracy: 0.5156 - val_loss: nan - val_accuracy: 0.4688 - lr: 0.1000
Epoch 6/30
6/6 [==============================] - 71s 12s/step - loss: nan - accuracy: 0.5156 - val_loss: nan - val_accuracy: 0.3750 - lr: 0.0100
Epoch 7/30
6/6 [==============================] - 60s 10s/step - loss: nan - accuracy: 0.4270 - val_loss: nan - val_accuracy

using alexnet

In [16]:
test_loss, test_accuracy = model2.evaluate(test_generator)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)

14/14 [==============================] - 63s 4s/step - loss: 0.1302 - accuracy: 0.9490
Test Loss: 0.13022984564304352
Test Accuracy: 0.9489558935165405


In [21]:
def preprocess_image(img_path, target_size=(224, 224)):
    
    if not isinstance(target_size, tuple):
        raise ValueError("target_size must be a tuple of two integers (height, width)")

    img = Image.open(img_path)
    img = img.resize(target_size)
    img_array = np.array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array

In [23]:
preprocessed_img = preprocess_image(img_path, target_size=(224, 224))

# Make predictions
predictions = model2.predict(preprocessed_img)
print(predictions)
# Get the predicted class index
predicted_class_index = np.argmax(predictions[0])
print(predicted_class_index)
# Get the class label associated with the predicted index
class_labels = list(train_generator.class_indices.keys())
predicted_class_label = class_labels[predicted_class_index]

print("Predicted Class Label:", predicted_class_label)

1/1 [==============================] - 0s 123ms/step
[[nan nan nan]]
0
Predicted Class Label: Potato___Early_blight


In [18]:
model2.save("pototo_model.h5")